## Train
---
- 사용 데이터셋
    - 4_Animal_Skin
    - 5_Animal_Eyes

In [ ]:
import os
import random
import shutil
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet34, ResNet34_Weights
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np
from collections import defaultdict
from sklearn.utils.class_weight import compute_class_weight

# =========================
# 0. 설정
# =========================
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

SKIN_ROOT = "files/4_Animal_Skin"
EYES_ROOT = "files/5_Animal_Eyes"
WORK_DIR = "files/work/disease_dataset"

BATCH_SIZE = 32  # 🔥 16 → 32 (속도 개선)
EPOCHS = 50
LR_INITIAL = 1e-4  # 🔥 Scheduler 사용
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
NUM_WORKERS = 16  # 🔥 8 → 16 (속도 개선)

# 🔥 Early Stopping
PATIENCE = 10
MIN_DELTA = 0.001

# 🔥 Mixup
# MIXUP_ALPHA = 0.4
MIXUP_ALPHA = 0

print(f"🎯 Device: {DEVICE}")
print(f"⚡ Batch Size: {BATCH_SIZE}, Workers: {NUM_WORKERS}")

# =========================
# 1. Dataset Preparation
# =========================
def collect_samples(root):
    samples = []
    for class_dir in sorted(os.listdir(root)):
        class_path = os.path.join(root, class_dir)
        if not os.path.isdir(class_path):
            continue
        for file in os.listdir(class_path):
            if file.lower().endswith(('.jpg','.png','.jpeg')):
                samples.append((class_dir, os.path.join(class_path, file)))
    print(f"  → {len(samples)} samples, {len(set(s[0] for s in samples))} classes")
    return samples

def split_and_copy(samples, task_name):
    random.shuffle(samples)
    class_samples = defaultdict(list)
    for label, path in samples:
        class_samples[label].append(path)

    for split in ["train","val","test"]:
        os.makedirs(os.path.join(WORK_DIR,split,task_name),exist_ok=True)

    for label, paths in class_samples.items():
        n = len(paths)
        n_train = int(n*0.8)
        n_val = int(n*0.1)

        splits = {
            "train": paths[:n_train],
            "val": paths[n_train:n_train+n_val],
            "test": paths[n_train+n_val:]
        }

        for split_name, split_paths in splits.items():
            for src in tqdm(split_paths, desc=f"{task_name}/{split_name}/{label}", leave=False):
                dst_dir = os.path.join(WORK_DIR,split_name,task_name,label)
                os.makedirs(dst_dir,exist_ok=True)
                shutil.copy(src, os.path.join(dst_dir, os.path.basename(src)))

def prepare_dataset():
    if os.path.exists(WORK_DIR):
        shutil.rmtree(WORK_DIR)

    print("\n📦 Collecting Skin Disease...")
    skin = collect_samples(SKIN_ROOT)
    print("  📋 Splitting & Copying...")
    split_and_copy(skin,"skin")

    print("\n📦 Collecting Eye Disease...")
    eyes = collect_samples(EYES_ROOT)
    print("  📋 Splitting & Copying...")
    split_and_copy(eyes,"eyes")

    print("\n✅ Dataset ready")

# =========================
# 2. Dataset Class (속도 최적화)
# =========================
class ImageDataset(Dataset):
    def __init__(self, task_dir, augment=False):
        self.samples = []
        self.label_to_id = {}

        for label in sorted(os.listdir(task_dir)):
            label_dir = os.path.join(task_dir,label)
            if not os.path.isdir(label_dir):
                continue
            self.label_to_id[label] = len(self.label_to_id)
            for file in os.listdir(label_dir):
                if file.lower().endswith(('.jpg','.png','.jpeg')):
                    self.samples.append((os.path.join(label_dir,file),label))

        print(f"  📊 {os.path.basename(task_dir)}: {len(self.samples)} samples, {len(self.label_to_id)} classes")

        if augment:
            self.transform = transforms.Compose([
                transforms.Resize((256,256)),
                transforms.RandomCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(15),  # 🔥 추가
                transforms.ColorJitter(0.3,0.3,0.3),  # 🔥 강화
                transforms.ToTensor(),
                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self,idx):
        path,label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        img = self.transform(img)
        return img, self.label_to_id[label]

# =========================
# 3. Disease Omni Model
# =========================
class DiseaseOmni(nn.Module):
    def __init__(self, num_skin, num_eyes):
        super().__init__()

        # Skin backbone
        skin_backbone = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
        in_features_s = skin_backbone.fc.in_features
        skin_backbone.fc = nn.Identity()
        self.skin_backbone = skin_backbone
        self.skin_head = nn.Sequential(  # 🔥 Dropout 추가
            nn.Dropout(0.3),
            nn.Linear(in_features_s, num_skin)
        )

        # Eyes backbone
        eyes_backbone = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
        in_features_e = eyes_backbone.fc.in_features
        eyes_backbone.fc = nn.Identity()
        self.eyes_backbone = eyes_backbone
        self.eyes_head = nn.Sequential(  # 🔥 Dropout 추가
            nn.Dropout(0.3),
            nn.Linear(in_features_e, num_eyes)
        )

    def forward(self,x,task):
        if task=="skin":
            feat = self.skin_backbone(x)
            return self.skin_head(feat)
        elif task=="eyes":
            feat = self.eyes_backbone(x)
            return self.eyes_head(feat)
        else:
            raise ValueError("Task must be 'skin' or 'eyes'")

# =========================
# 🔥 4. Mixup Function
# =========================
def mixup_data(x, y, alpha=0.4):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    
    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# =========================
# 🔥 5. Early Stopping Class
# =========================
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        
    def __call__(self, val_acc):
        score = val_acc
        
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            print(f"  ⚠️  EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

# =========================
# 6. Training (완전 개선)
# =========================
def train():
    prepare_dataset()

    print("\n🔄 Loading datasets...")
    skin_train = ImageDataset(os.path.join(WORK_DIR,"train","skin"), augment=True)
    skin_val   = ImageDataset(os.path.join(WORK_DIR,"val","skin"), augment=False)

    eyes_train = ImageDataset(os.path.join(WORK_DIR,"train","eyes"), augment=True)
    eyes_val   = ImageDataset(os.path.join(WORK_DIR,"val","eyes"), augment=False)

    # 🔥 persistent_workers=True (속도 개선)
    skin_train_loader = DataLoader(
        skin_train, BATCH_SIZE, True, 
        num_workers=NUM_WORKERS, pin_memory=True, 
        persistent_workers=True, prefetch_factor=2
    )
    skin_val_loader = DataLoader(
        skin_val, BATCH_SIZE, False, 
        num_workers=NUM_WORKERS//2, pin_memory=True
    )

    eyes_train_loader = DataLoader(
        eyes_train, BATCH_SIZE, True, 
        num_workers=NUM_WORKERS, pin_memory=True, 
        persistent_workers=True, prefetch_factor=2
    )
    eyes_val_loader = DataLoader(
        eyes_val, BATCH_SIZE, False, 
        num_workers=NUM_WORKERS//2, pin_memory=True
    )

    print(f"\n📦 DataLoaders:")
    print(f"  Skin: {len(skin_train_loader)} train batches, {len(skin_val_loader)} val batches")
    print(f"  Eyes: {len(eyes_train_loader)} train batches, {len(eyes_val_loader)} val batches")

    # 🔥 클래스 가중치 계산
    print("\n⚖️  Computing class weights...")
    skin_labels = [skin_train.label_to_id[label] for _, label in skin_train.samples]
    skin_weights = compute_class_weight('balanced', classes=np.arange(len(skin_train.label_to_id)), y=skin_labels)
    skin_weights_tensor = torch.tensor(skin_weights, dtype=torch.float).to(DEVICE)

    eyes_labels = [eyes_train.label_to_id[label] for _, label in eyes_train.samples]
    eyes_weights = compute_class_weight('balanced', classes=np.arange(len(eyes_train.label_to_id)), y=eyes_labels)
    eyes_weights_tensor = torch.tensor(eyes_weights, dtype=torch.float).to(DEVICE)

    print(f"  Skin: weights range [{skin_weights.min():.2f}, {skin_weights.max():.2f}]")
    print(f"  Eyes: weights range [{eyes_weights.min():.2f}, {eyes_weights.max():.2f}]")

    model = DiseaseOmni(
        len(skin_train.label_to_id),
        len(eyes_train.label_to_id)
    ).to(DEVICE)

    # optimizer = torch.optim.AdamW(model.parameters(), lr=LR_INITIAL, weight_decay=0.01)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR_INITIAL, weight_decay=0)
    scaler = torch.amp.GradScaler("cuda")

    # 🔥 Scheduler 변경: 안정적인 Cosine Decay
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=EPOCHS, eta_min=1e-6
    )
    
    # 🔥 클래스 가중치 적용
    # criterion_skin = nn.CrossEntropyLoss(weight=skin_weights_tensor, label_smoothing=0.1)
    # criterion_eyes = nn.CrossEntropyLoss(weight=eyes_weights_tensor, label_smoothing=0.1)
    criterion_skin = nn.CrossEntropyLoss(weight=skin_weights_tensor, label_smoothing=0)
    criterion_eyes = nn.CrossEntropyLoss(weight=eyes_weights_tensor, label_smoothing=0)
    
    # 🔥 Early Stopping
    early_stopping = EarlyStopping(patience=PATIENCE, min_delta=MIN_DELTA)

    best_avg_acc = 0
    history = []

    for epoch in range(EPOCHS):
        print(f"\n{'='*60}")
        print(f"Epoch {epoch+1}/{EPOCHS} | LR: {optimizer.param_groups[0]['lr']:.2e}")
        print(f"{'='*60}")
        
        model.train()
        loss_s_total, loss_e_total = 0, 0

        # -------- Skin Disease --------
        print(f"\n🩺 Training Skin Disease...")
        for imgs, labels in tqdm(skin_train_loader, desc="Skin", leave=False):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

            # 🔥 Mixup
            imgs, labels_a, labels_b, lam = mixup_data(imgs, labels, MIXUP_ALPHA)

            optimizer.zero_grad()
            
            with torch.amp.autocast("cuda"):
                logits = model(imgs, "skin")
                loss = mixup_criterion(criterion_skin, logits, labels_a, labels_b, lam)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            loss_s_total += loss.item()

        avg_loss_s = loss_s_total / len(skin_train_loader)
        print(f"  → Avg Loss: {avg_loss_s:.4f}")

        # -------- Eye Disease --------
        print(f"\n👁️  Training Eye Disease...")
        for imgs, labels in tqdm(eyes_train_loader, desc="Eyes", leave=False):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

            # 🔥 Mixup
            imgs, labels_a, labels_b, lam = mixup_data(imgs, labels, MIXUP_ALPHA)

            optimizer.zero_grad()
            
            with torch.amp.autocast("cuda"):
                logits = model(imgs, "eyes")
                loss = mixup_criterion(criterion_eyes, logits, labels_a, labels_b, lam)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            loss_e_total += loss.item()

        avg_loss_e = loss_e_total / len(eyes_train_loader)
        print(f"  → Avg Loss: {avg_loss_e:.4f}")

        # 🔥 Scheduler step
        scheduler.step()

        # -------- Validation --------
        print(f"\n🔍 Validation...")
        model.eval()
        correct_s, total_s = 0, 0
        correct_e, total_e = 0, 0

        with torch.no_grad():
            for imgs, labels in tqdm(skin_val_loader, desc="Val Skin", leave=False):
                imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                pred = model(imgs, "skin").argmax(-1)
                correct_s += (pred == labels).sum().item()
                total_s += labels.size(0)

            for imgs, labels in tqdm(eyes_val_loader, desc="Val Eyes", leave=False):
                imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                pred = model(imgs, "eyes").argmax(-1)
                correct_e += (pred == labels).sum().item()
                total_e += labels.size(0)

        acc_s = correct_s / total_s
        acc_e = correct_e / total_e
        avg_acc = (acc_s + acc_e) / 2

        print(f"\n📊 Results:")
        print(f"  Skin: Loss {avg_loss_s:.4f} | Acc {acc_s:.4f} ({acc_s*100:.1f}%)")
        print(f"  Eyes: Loss {avg_loss_e:.4f} | Acc {acc_e:.4f} ({acc_e*100:.1f}%)")
        print(f"  Average Acc: {avg_acc:.4f} ({avg_acc*100:.1f}%)")

        # History 저장
        history.append({
            'epoch': epoch+1,
            'lr': optimizer.param_groups[0]['lr'],
            'loss_s': avg_loss_s,
            'loss_e': avg_loss_e,
            'acc_s': acc_s,
            'acc_e': acc_e,
            'acc_avg': avg_acc
        })

        # Best 모델 저장
        if avg_acc > best_avg_acc:
            best_avg_acc = avg_acc
            torch.save({
                "model": model.state_dict(),
                "skin_label_to_id": skin_train.label_to_id,
                "eyes_label_to_id": eyes_train.label_to_id,
                "best_epoch": epoch+1,
                "best_acc": best_avg_acc,
                "history": history
            }, "pet_disease_omni_best.pth")
            print(f"  💾 Saved new best model! (Acc: {best_avg_acc:.4f})")

        # 🔥 Early Stopping 체크
        early_stopping(avg_acc)
        if early_stopping.early_stop:
            print(f"\n🛑 Early stopping triggered at epoch {epoch+1}")
            break

    # 그래프
    print("\n📈 Generating training history plot...")
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Accuracy
    axes[0, 0].plot([h['acc_s'] for h in history], 'b-', label='Skin', linewidth=2)
    axes[0, 0].plot([h['acc_e'] for h in history], 'r-', label='Eyes', linewidth=2)
    axes[0, 0].plot([h['acc_avg'] for h in history], 'g--', label='Average', linewidth=2)
    axes[0, 0].set_xlabel('Epoch'); axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].set_title('Accuracy'); axes[0, 0].set_ylim(0, 1)
    axes[0, 0].grid(True, alpha=0.3); axes[0, 0].legend()

    # Loss
    axes[0, 1].plot([h['loss_s'] for h in history], 'b-', label='Skin', linewidth=2)
    axes[0, 1].plot([h['loss_e'] for h in history], 'r-', label='Eyes', linewidth=2)
    axes[0, 1].set_xlabel('Epoch'); axes[0, 1].set_ylabel('Loss')
    axes[0, 1].set_title('Training Loss')
    axes[0, 1].grid(True, alpha=0.3); axes[0, 1].legend()

    # Learning Rate
    axes[1, 0].plot([h['lr'] for h in history], 'purple', linewidth=2)
    axes[1, 0].set_xlabel('Epoch'); axes[1, 0].set_ylabel('Learning Rate')
    axes[1, 0].set_title('Learning Rate Schedule')
    axes[1, 0].set_yscale('log'); axes[1, 0].grid(True, alpha=0.3)

    # Average Accuracy (확대)
    axes[1, 1].plot([h['acc_avg'] for h in history], 'g-', linewidth=3)
    axes[1, 1].axhline(y=best_avg_acc, color='r', linestyle='--', label=f'Best: {best_avg_acc:.4f}')
    axes[1, 1].set_xlabel('Epoch'); axes[1, 1].set_ylabel('Accuracy')
    axes[1, 1].set_title('Average Accuracy')
    axes[1, 1].set_ylim(0, 1); axes[1, 1].grid(True, alpha=0.3); axes[1, 1].legend()

    plt.tight_layout()
    plt.savefig('pet_disease_history.png', dpi=150, bbox_inches='tight')
    print("  ✅ Saved: pet_disease_history.png")

    print(f"\n🎉 Training Finished!")
    print(f"  Best Average Acc: {best_avg_acc:.4f} ({best_avg_acc*100:.1f}%)")
    print(f"  Total Epochs: {len(history)}")

if __name__ == "__main__":
    train()

🎯 Device: cuda:1
⚡ Batch Size: 32, Workers: 16

📦 Collecting Skin Disease...
  → 50000 samples, 10 classes
  📋 Splitting & Copying...



📦 Collecting Eye Disease...
  → 133041 samples, 21 classes
  📋 Splitting & Copying...



✅ Dataset ready

🔄 Loading datasets...
  📊 skin: 40000 samples, 10 classes
  📊 skin: 5000 samples, 10 classes
  📊 eyes: 106428 samples, 21 classes
  📊 eyes: 13299 samples, 21 classes

📦 DataLoaders:
  Skin: 1250 train batches, 157 val batches
  Eyes: 3326 train batches, 416 val batches

⚖️  Computing class weights...
  Skin: weights range [1.00, 1.00]
  Eyes: weights range [0.16, 6.67]

Epoch 1/50 | LR: 1.00e-04

🩺 Training Skin Disease...


  → Avg Loss: 1.8899

👁️  Training Eye Disease...


  → Avg Loss: 2.5695

🔍 Validation...


...

  → Avg Loss: 1.2249

👁️  Training Eye Disease...


  → Avg Loss: 1.9157

🔍 Validation...



📊 Results:
  Skin: Loss 1.2249 | Acc 0.6296 (63.0%)
  Eyes: Loss 1.9157 | Acc 0.6546 (65.5%)
  Average Acc: 0.6421 (64.2%)
  ⚠️  EarlyStopping counter: 9/10

Epoch 40/50 | LR: 5.98e-05

🩺 Training Skin Disease...


  → Avg Loss: 1.2140

👁️  Training Eye Disease...


KeyboardInterrupt: 

## Test

In [ ]:
# import os
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# import torchvision.transforms as transforms
# from torchvision.models import resnet34, ResNet34_Weights
# from PIL import Image
# import numpy as np
# from tqdm import tqdm
# from collections import defaultdict
# import matplotlib.pyplot as plt
# from sklearn.metrics import classification_report, confusion_matrix
# import seaborn as sns

# # =========================
# # 모델 클래스 (train과 동일)
# # =========================
# class AbnormalityOmni(nn.Module):
#     def __init__(self, num_classes_per_task):
#         super().__init__()
#         self.trunk = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
#         in_features = self.trunk.fc.in_features
#         self.trunk.fc = nn.Identity()
        
#         self.pet_head = nn.Linear(in_features, 2)  # dog/cat
        
#         self.task_heads = nn.ModuleDict({
#             task: nn.Linear(in_features, num_classes) 
#             for task, num_classes in num_classes_per_task.items()
#         })
    
#     def forward(self, x, task):
#         if x.dim() == 5:  # 1x5x3x224x224
#             B = x.size(0)
#             x = x.view(-1, *x.shape[2:])
        
#         feats = self.trunk(x)
#         if x.size(0) != feats.size(0):
#             feats = feats.view(B, -1, feats.size(-1)).mean(1)
        
#         pet_logits = self.pet_head(feats)
#         task_logits = self.task_heads[task](feats)
        
#         return {
#             'pet': F.softmax(pet_logits, dim=-1),
#             task: task_logits
#         }

# # =========================
# # Test Dataset (train 의존성 제거)
# # =========================
# class TestImageDataset(Dataset):
#     def __init__(self, task_dir, label_to_id):
#         self.samples = []
#         self.id_to_label = {v: k for k, v in label_to_id.items()}
        
#         for label in os.listdir(task_dir):
#             if label not in label_to_id:
#                 continue
#             label_dir = os.path.join(task_dir, label)
#             for file in os.listdir(label_dir):
#                 if file.lower().endswith(('.jpg', '.png', '.jpeg')):
#                     self.samples.append((
#                         os.path.join(label_dir, file),
#                         label_to_id[label]
#                     ))
        
#         print(f"  📊 Test {os.path.basename(task_dir)}: {len(self.samples)} samples")
        
#         self.transform = transforms.Compose([
#             transforms.Resize((224, 224)),
#             transforms.ToTensor(),
#             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#         ])
    
#     def __len__(self):
#         return len(self.samples)
    
#     def __getitem__(self, idx):
#         path, label_id = self.samples[idx]
#         img = Image.open(path).convert("RGB")
#         img = self.transform(img)
#         return img, label_id

# # =========================
# # Test 함수
# # =========================
# def test():
#     DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
#     BATCH_SIZE = 32  # Test는 크게
    
#     print("🔎 Loading best model...")
#     checkpoint = torch.load("pet_abnormality_omni_best.pth", map_location=DEVICE)
    
#     num_classes_per_task = checkpoint["num_classes_per_task"]
#     label_to_id = checkpoint["label_to_id"]
    
#     # 모델 복원
#     model = AbnormalityOmni(num_classes_per_task).to(DEVICE)
#     model.load_state_dict(checkpoint["model"])
#     model.eval()
    
#     print("📦 Loading TEST datasets...")
#     TEST_DIR = os.path.join("files", "work", "abnormality_omni", "test")
#     tasks = ["skin", "eye"]
    
#     test_loaders = {}
#     for task in tasks:
#         test_loaders[task] = DataLoader(
#             TestImageDataset(
#                 os.path.join(TEST_DIR, task),
#                 label_to_id[task]
#             ),
#             BATCH_SIZE, False, num_workers=8, pin_memory=True
#         )
    
#     # =========================
#     # Evaluation
#     # =========================
#     results = {}
#     all_preds = {}
#     all_labels = {}
    
#     for task in tasks:
#         print(f"\n🔍 Testing {task.upper()}...")
#         correct, total = 0, 0
#         all_preds[task] = []
#         all_labels[task] = []
        
#         with torch.no_grad():
#             for imgs, labels in tqdm(test_loaders[task], desc=f"Test {task}"):
#                 imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                
#                 outputs = model(imgs, task)
#                 pred = outputs[task].argmax(dim=-1)
                
#                 correct += (pred == labels).sum().item()
#                 total += labels.size(0)
                
#                 all_preds[task].extend(pred.cpu().numpy())
#                 all_labels[task].extend(labels.cpu().numpy())
        
#         acc = correct / total
#         results[task] = acc
#         print(f"  → Accuracy: {acc:.4f} ({acc*100:.1f}%)")
    
#     # Average
#     avg_acc = sum(results.values()) / len(tasks)
#     print(f"\n📊 FINAL TEST Results:")
#     for task, acc in results.items():
#         print(f"  {task.capitalize()}: {acc:.4f} ({acc*100:.1f}%)")
#     print(f"  Average: {avg_acc:.4f} ({avg_acc*100:.1f}%)")
    
#     # =========================
#     # Detailed Metrics + Confusion Matrix
#     # =========================
#     plt.figure(figsize=(15, 5))
    
#     for i, task in enumerate(tasks):
#         plt.subplot(1, len(tasks), i+1)
        
#         # Classification Report
#         print(f"\n{task.upper()} Classification Report:")
#         print(classification_report(
#             all_labels[task], all_preds[task], 
#             target_names=list(label_to_id[task].keys()),
#             digits=4
#         ))
        
#         # Confusion Matrix
#         cm = confusion_matrix(all_labels[task], all_preds[task])
#         sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
#         plt.title(f'{task.capitalize()} Confusion Matrix')
#         plt.ylabel('True'); plt.xlabel('Pred')
    
#     plt.tight_layout()
#     plt.savefig('abnormality_test_results.png', dpi=150, bbox_inches='tight')
#     print("✅ Saved: abnormality_test_results.png")
    
#     # Pet prediction accuracy (bonus)
#     print("\n🔍 Pet (dog/cat) Accuracy Check...")
#     pet_correct, pet_total = 0, 0
#     # 전체 test에서 pet 예측 샘플링 체크 (실제 app처럼)
#     with torch.no_grad():
#         for task in tasks:
#             for imgs, _ in test_loaders[task][:5]:  # 샘플
#                 imgs = imgs.to(DEVICE)
#                 pet_probs = model(imgs[:1], task)['pet']  # single img test
#                 pet_pred = pet_probs.argmax().item()
#                 pet_total += 1
#                 # 실제 pet 라벨 없으므로 confidence만
#                 pet_correct += float(pet_probs.max() > 0.8)
    
#     pet_acc = pet_correct / pet_total
#     print(f"  Pet classification conf>0.8: {pet_acc:.4f}")
    
#     # Save results
#     torch.save({
#         'test_results': results,
#         'avg_acc': avg_acc,
#         'predictions': all_preds,
#         'labels': all_labels
#     }, "abnormality_test_results.pth")

# if __name__ == "__main__":
#     test()